In [1]:
import pandas as pd
import requests
import json
import pandas as pd

In [2]:
communities_dicc= {
                8742:'canarias', 
                8743:'baleares',
                8744:'ceuta',
                8745:'melilla',
                4:'Andalucía',
                5:'Aragón',
                6:'Cantabria',
                7:'Castilla_la_Mancha',
                8:'Castilla_León',
                9:'Cataluña',
                10:'País_Vasco',
                11:'Asturias',
                8744:'Ceuta',
                8745:'Melilla',
                13:'Madrid',
                14:'Navarra', 
                15:'Valenciana',
                16:'Extremadura', 
                17:'Galicia',
                8743:'Baleares',
                8742:'Canarias',
                20:'La_Rioja',
                21:'Murcia',
                }

In [3]:
def ree_query(start_date, end_date, ccaa_code):
    headers = {'Accept': 'application/json',
               'Content-Type': 'applic<ation/json',
               'Host': 'apidatos.ree.es'}
    url= f"https://apidatos.ree.es/es/datos/generacion/estructura-generacion?start_date={start_date}T00:00&end_date={end_date}T23:59&time_trunc=month&geo_trunc=electric_system&geo_limit=ccaa&geo_ids={ccaa_code}"    
    answer = requests.get(url, headers).json()
    res = answer['included']
    answer_list = []
    for i in res:

        fuente = i['attributes']['title']
        tipo = i['attributes']['type']
        fecha = pd.to_datetime(i['attributes']['values'][0]['datetime']).date()
        value = i['attributes']['values'][0]['value']
        percentage = i['attributes']['values'][0]['percentage']

        small_dict = {
        "class": fuente,
        "type": tipo,
        "date" : fecha,
        "value" : value,
        "percentage": percentage,
         "aacc":communities_dicc[ccaa_code]}

        answer_list.append(small_dict)
        answer_list
        df=pd.DataFrame(answer_list)
    return df

In [4]:
def ree_yearly(): 
    years =[2014,2015,2016,2017,2018,2019,2020,2021,2022,2023]
    years_df = []
    for i in years:
        lista_df=[]
        for community in communities_dicc:
            ccaa_code = community
            df = ree_query(f"{i}-01-01", f"{i+1}-01-01", ccaa_code)
            lista_df.append(df)
        lista_df = pd.concat(lista_df, ignore_index=True) 
        years_df.append(lista_df)
    final_df = pd.concat(years_df, ignore_index=True) 
    return final_df

In [5]:
years =[2014,2015,2016,2017,2018,2019,2020,2021,2022,2023]
print(years[:-1])


[2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]


In [6]:
df = ree_yearly()

In [7]:
df

,class,type,date,value,percentage,aacc
0,Hidráulica,Renovable,2014-01-01,303.396,0.000421,Canarias
1,Motores diésel,No-Renovable,2014-01-01,181768.586,0.251957,Canarias
2,Turbina de gas,No-Renovable,2014-01-01,26245.404,0.036380,Canarias
3,Turbina de vapor,No-Renovable,2014-01-01,216697.507,0.300373,Canarias
4,Ciclo combinado,No-Renovable,2014-01-01,247173.544,0.342618,Canarias
...,...,...,...,...,...,...
1687,Solar fotovoltaica,Renovable,2023-01-01,136319.048,0.224809,Murcia
1688,Solar térmica,Renovable,2023-01-01,1259.723,0.002077,Murcia
1689,Otras renovables,Renovable,2023-01-01,2356.086,0.003886,Murcia
1690,Cogeneración,No-Renovable,2023-01-01,76614.473,0.126348,Murcia


In [8]:
df["comunidad"] = df["aacc"].map(communities_dicc)

In [11]:
df.drop(columns = 'comunidad', inplace=True)

In [12]:
df

,class,type,date,value,percentage,aacc
0,Hidráulica,Renovable,2014-01-01,303.396,0.000421,Canarias
1,Motores diésel,No-Renovable,2014-01-01,181768.586,0.251957,Canarias
2,Turbina de gas,No-Renovable,2014-01-01,26245.404,0.036380,Canarias
3,Turbina de vapor,No-Renovable,2014-01-01,216697.507,0.300373,Canarias
4,Ciclo combinado,No-Renovable,2014-01-01,247173.544,0.342618,Canarias
...,...,...,...,...,...,...
1687,Solar fotovoltaica,Renovable,2023-01-01,136319.048,0.224809,Murcia
1688,Solar térmica,Renovable,2023-01-01,1259.723,0.002077,Murcia
1689,Otras renovables,Renovable,2023-01-01,2356.086,0.003886,Murcia
1690,Cogeneración,No-Renovable,2023-01-01,76614.473,0.126348,Murcia


In [13]:
df.to_csv("data/df_generated_monthly_ccaa.csv")

In [14]:
df['date'].max()

datetime.date(2023, 3, 1)

In [43]:
answer['included'][0]['attributes']['values'][0]['datetime']

'2014-01-01T00:00:00.000+01:00'